In [1]:
! pip install geopandas
! pip install pandas
! pip install pygeos
! pip install boto
! pip install s3fs
! pip install shapely
! pip install gcsfs
!pip install contextily
!pip install matplotlib-scalebar
!pip install sklearn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import time
from itertools import groupby
import geopandas as gpd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.ticker as ticker
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_1802928/1227132047.py:18: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html)

In [2]:
cordon_zones = ['060750615006',
'060750615005',
'060750615004',
'060750615003',
'060750615002',
'060750615001',
'060750611003',
'060750611002',
'060750611001',
'060750607003',
'060750607002',
'060750607001',
'060750202001',
'060750201004',
'060750201003',
'060750201002',
'060750201001',
'060750180002',
'060750180001',
'060750178022',
'060750178021',
'060750178012',
'060750178011',
'060750177002',
'060750177001',
'060750176015',
'060750176014',
'060750176013',
'060750176012',
'060750176011',
'060750168023',
'060750168021',
'060750162003',
'060750162002',
'060750162001',
'060750125022',
'060750125021',
'060750125012',
'060750125011',
'060750124023',
'060750124022',
'060750124021',
'060750124012',
'060750124011',
'060750123022',
'060750123021',
'060750123012',
'060750123011',
'060750122021',
'060750122012',
'060750122011',
'060750121002',
'060750121001',
'060750120002',
'060750120001',
'060750119022',
'060750119021',
'060750119012',
'060750119011',
'060750118001',
'060750117002',
'060750117001',
'060750113002',
'060750113001',
'060750112003',
'060750112002',
'060750112001',
'060750111003',
'060750111002',
'060750111001',
'060750110003',
'060750110002',
'060750110001',
'060750109003',
'060750109002',
'060750109001',
'060750108003',
'060750108002',
'060750108001',
'060750107004',
'060750107003',
'060750107002',
'060750107001',
'060750106003',
'060750106002',
'060750106001',
'060750105002',
'060750105001',
'060750104004',
'060750104003',
'060750104002',
'060750104001',
'060750103003',
'060750103002',
'060750103001',
'060750102003',
'060750102002',
'060750102001',
'060750101002',
'060750101001']

SF_zones = ['060750101001',
'060750101002',
'060750102001',
'060750102002',
'060750102003',
'060750103001',
'060750103002',
'060750103003',
'060750104001',
'060750104002',
'060750104003',
'060750104004',
'060750105001',
'060750105002',
'060750106001',
'060750106002',
'060750106003',
'060750107001',
'060750107002',
'060750107003',
'060750107004',
'060750108001',
'060750108002',
'060750108003',
'060750109001',
'060750109002',
'060750109003',
'060750110001',
'060750110002',
'060750110003',
'060750111001',
'060750111002',
'060750111003',
'060750112001',
'060750112002',
'060750112003',
'060750113001',
'060750113002',
'060750117001',
'060750117002',
'060750118001',
'060750119011',
'060750119012',
'060750119021',
'060750119022',
'060750120001',
'060750120002',
'060750121001',
'060750121002',
'060750122011',
'060750122012',
'060750122021',
'060750123011',
'060750123012',
'060750123021',
'060750123022',
'060750124011',
'060750124012',
'060750124021',
'060750124022',
'060750124023',
'060750125011',
'060750125012',
'060750125021',
'060750125022',
'060750126011',
'060750126021',
'060750126022',
'060750127001',
'060750127002',
'060750127003',
'060750128001',
'060750128002',
'060750128003',
'060750128004',
'060750129011',
'060750129012',
'060750129021',
'060750129022',
'060750129023',
'060750130001',
'060750130002',
'060750130003',
'060750130004',
'060750131011',
'060750131012',
'060750131021',
'060750131022',
'060750132001',
'060750132002',
'060750132003',
'060750133001',
'060750133002',
'060750133003',
'060750133004',
'060750133005',
'060750134001',
'060750134002',
'060750134003',
'060750135001',
'060750135002',
'060750151001',
'060750151002',
'060750152001',
'060750152002',
'060750152003',
'060750153001',
'060750153002',
'060750154001',
'060750154002',
'060750154003',
'060750154004',
'060750154005',
'060750155001',
'060750155002',
'060750155003',
'060750156001',
'060750156002',
'060750156003',
'060750157001',
'060750157002',
'060750157003',
'060750157004',
'060750158011',
'060750158012',
'060750158013',
'060750158021',
'060750158022',
'060750159001',
'060750159002',
'060750160001',
'060750161001',
'060750161002',
'060750161003',
'060750161004',
'060750162001',
'060750162002',
'060750162003',
'060750163001',
'060750163002',
'060750163003',
'060750164001',
'060750164002',
'060750165001',
'060750165002',
'060750165003',
'060750165004',
'060750166001',
'060750166002',
'060750166003',
'060750166004',
'060750167001',
'060750167002',
'060750167003',
'060750167004',
'060750168011',
'060750168012',
'060750168013',
'060750168021',
'060750168022',
'060750168023',
'060750169001',
'060750169002',
'060750170001',
'060750170002',
'060750170003',
'060750171011',
'060750171012',
'060750171013',
'060750171021',
'060750171022',
'060750171023',
'060750176011',
'060750176012',
'060750176013',
'060750176014',
'060750176015',
'060750177001',
'060750177002',
'060750178011',
'060750178012',
'060750178021',
'060750178022',
'060750180001',
'060750180002',
'060750201001',
'060750201002',
'060750201003',
'060750201004',
'060750202001',
'060750202002',
'060750202003',
'060750203001',
'060750203002',
'060750203003',
'060750204011',
'060750204012',
'060750204013',
'060750204021',
'060750204022',
'060750205001',
'060750205002',
'060750205003',
'060750206001',
'060750206002',
'060750206003',
'060750206004',
'060750207001',
'060750207002',
'060750207003',
'060750208001',
'060750208002',
'060750208003',
'060750208004',
'060750209001',
'060750209002',
'060750209003',
'060750209004',
'060750210001',
'060750210002',
'060750210003',
'060750210004',
'060750211001',
'060750211002',
'060750211003',
'060750211004',
'060750212001',
'060750212002',
'060750212003',
'060750213001',
'060750213002',
'060750214001',
'060750214002',
'060750214003',
'060750215001',
'060750215002',
'060750215003',
'060750215004',
'060750215005',
'060750216001',
'060750216002',
'060750217001',
'060750217002',
'060750217003',
'060750218001',
'060750218002',
'060750218003',
'060750218004',
'060750226001',
'060750226002',
'060750227021',
'060750227022',
'060750227041',
'060750227042',
'060750228011',
'060750228012',
'060750228013',
'060750228021',
'060750228022',
'060750228031',
'060750228032',
'060750228033',
'060750229011',
'060750229012',
'060750229013',
'060750229021',
'060750229022',
'060750229031',
'060750229032',
'060750229033',
'060750230011',
'060750230012',
'060750230013',
'060750230031',
'060750230032',
'060750231021',
'060750231022',
'060750231031',
'060750231032',
'060750232001',
'060750232002',
'060750232003',
'060750233001',
'060750234001',
'060750234002',
'060750251001',
'060750251002',
'060750251003',
'060750252001',
'060750252002',
'060750252003',
'060750252004',
'060750253001',
'060750253002',
'060750253003',
'060750253004',
'060750254011',
'060750254012',
'060750254013',
'060750254021',
'060750254022',
'060750254023',
'060750254031',
'060750254032',
'060750255001',
'060750255002',
'060750255003',
'060750255004',
'060750255005',
'060750255006',
'060750256001',
'060750256002',
'060750256003',
'060750256004',
'060750257011',
'060750257012',
'060750257013',
'060750257021',
'060750257022',
'060750257023',
'060750258001',
'060750258002',
'060750259001',
'060750259002',
'060750259003',
'060750260011',
'060750260012',
'060750260021',
'060750260022',
'060750260031',
'060750260032',
'060750260041',
'060750260042',
'060750261001',
'060750261002',
'060750261003',
'060750261004',
'060750262001',
'060750262002',
'060750262003',
'060750262004',
'060750262005',
'060750263011',
'060750263012',
'060750263013',
'060750263021',
'060750263022',
'060750263023',
'060750263031',
'060750263032',
'060750264011',
'060750264012',
'060750264021',
'060750264022',
'060750264023',
'060750264031',
'060750264032',
'060750264041',
'060750264042',
'060750301011',
'060750301012',
'060750301013',
'060750301014',
'060750301021',
'060750301022',
'060750301023',
'060750302011',
'060750302012',
'060750302013',
'060750302021',
'060750302022',
'060750302023',
'060750303011',
'060750303012',
'060750303013',
'060750303014',
'060750303021',
'060750303022',
'060750303023',
'060750304001',
'060750304002',
'060750304003',
'060750304004',
'060750304005',
'060750305001',
'060750305002',
'060750305003',
'060750306001',
'060750306002',
'060750306003',
'060750307001',
'060750307002',
'060750307003',
'060750308001',
'060750308002',
'060750308003',
'060750308004',
'060750308005',
'060750309001',
'060750309002',
'060750309003',
'060750309004',
'060750309005',
'060750309006',
'060750309007',
'060750310001',
'060750310002',
'060750310003',
'060750311001',
'060750311002',
'060750311003',
'060750311004',
'060750311005',
'060750312011',
'060750312012',
'060750312013',
'060750312014',
'060750312021',
'060750312022',
'060750313011',
'060750313012',
'060750313013',
'060750313021',
'060750313022',
'060750313023',
'060750314001',
'060750314002',
'060750314003',
'060750314004',
'060750314005',
'060750326011',
'060750326012',
'060750326013',
'060750326021',
'060750326022',
'060750326023',
'060750327001',
'060750327002',
'060750327003',
'060750327004',
'060750327005',
'060750327006',
'060750327007',
'060750328011',
'060750328012',
'060750328013',
'060750328021',
'060750328022',
'060750328023',
'060750329011',
'060750329012',
'060750329013',
'060750329014',
'060750329021',
'060750329022',
'060750329023',
'060750330001',
'060750330002',
'060750330003',
'060750330004',
'060750330005',
'060750330006',
'060750331001',
'060750331002',
'060750331003',
'060750331004',
'060750332011',
'060750332031',
'060750332032',
'060750332041',
'060750332042',
'060750332043',
'060750351001',
'060750351002',
'060750351003',
'060750351004',
'060750351005',
'060750351006',
'060750351007',
'060750352011',
'060750352012',
'060750352013',
'060750352014',
'060750352015',
'060750352021',
'060750352022',
'060750352023',
'060750353001',
'060750353002',
'060750353003',
'060750353004',
'060750353005',
'060750353006',
'060750354001',
'060750354002',
'060750354003',
'060750354004',
'060750354005',
'060750401001',
'060750401002',
'060750401003',
'060750401004',
'060750402001',
'060750402002',
'060750402003',
'060750402004',
'060750426011',
'060750426012',
'060750426021',
'060750426022',
'060750426023',
'060750427001',
'060750427002',
'060750427003',
'060750428001',
'060750428002',
'060750428003',
'060750451001',
'060750451002',
'060750451003',
'060750452001',
'060750452002',
'060750452003',
'060750452004',
'060750452005',
'060750476001',
'060750476002',
'060750476003',
'060750476004',
'060750477011',
'060750477012',
'060750477013',
'060750477021',
'060750477022',
'060750477023',
'060750478011',
'060750478012',
'060750478013',
'060750478021',
'060750478022',
'060750478023',
'060750479011',
'060750479012',
'060750479013',
'060750479014',
'060750479015',
'060750479021',
'060750479022',
'060750479023',
'060750601001',
'060750604001',
'060750605021',
'060750605022',
'060750605023',
'060750607001',
'060750607002',
'060750607003',
'060750610001',
'060750610002',
'060750611001',
'060750611002',
'060750611003',
'060750612001',
'060750612002',
'060750614001',
'060750614002',
'060750614003',
'060750615001',
'060750615002',
'060750615003',
'060750615004',
'060750615005',
'060750615006',
'060759802001',
'060759803001',
'060759805011',
'060759806001',
'060759809001',
'060816004012',
'060816007001']

In [3]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:32610'): 
    gdf.crs = {'init': 'epsg:4326'}
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    # df.drop(columns=[xcol, ycol], inplace=True)
    
    return df.loc[~df.index.duplicated(keep='first'), :]

In [4]:
def addGeometryIdToDataFrame2(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:32610'): 
    gdf.crs = {'init': 'epsg:32610'}
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    # df.drop(columns=[xcol, ycol], inplace=True)
    
    return df.loc[~df.index.duplicated(keep='first'), :]

In [5]:

def process(plans, events_base, BGs, zones):

    #ADD ORIGIN AND DESTINATION COORDINATES TO PLANS 
    plans = plans[plans['planSelected']==True]
    plans_orig_to_des = plans.sort_values(by=['personId','planElementIndex'], ascending=True)
    print(plans_orig_to_des)

    xs = []
    ys = []
    starts = []
    for x, y, start in zip(plans_orig_to_des['activityLocationX'], plans_orig_to_des['activityLocationY'],plans_orig_to_des['activityEndTime']):
        if x > -999999999999999999:
            xs.append(x)
            ys.append(y)
            starts.append(0)
            current_start = start
            current_x = x
            current_y = y
        else:
            xs.append(current_x)
            ys.append(current_y)           
            starts.append(current_start)
            
    plans_orig_to_des['tripOriginX'] = xs
    plans_orig_to_des['tripOriginY'] = ys
    plans_orig_to_des['legDepartureTime'] = starts
    
    plans_orig_to_des = plans_orig_to_des.sort_values(by=['personId','planElementIndex'], ascending=False)
    
    xs = []
    ys = []
    for x, y in zip(plans_orig_to_des['activityLocationX'], plans_orig_to_des['activityLocationY']):
        if x > -999999999999999999:
            xs.append(x)
            ys.append(y)
            current_x = x
            current_y = y
        else:
            xs.append(current_x)
            ys.append(current_y)
    plans_orig_to_des['tripDestX'] = xs
    plans_orig_to_des['tripDestY'] = ys

    plans_orig_to_des = plans_orig_to_des.sort_values(by=['personId','planElementIndex'], ascending=True)
    
    plans_orig_to_des.dropna(subset=['legMode'], inplace=True)

    
    #ADD BLOCK ID TO PLANS

    plans_orig_to_des = addGeometryIdToDataFrame(plans_orig_to_des, BGs, 'tripOriginX', 'tripOriginY', 'ZIPo')
    plans_orig_to_des = addGeometryIdToDataFrame(plans_orig_to_des, BGs, 'tripDestX', 'tripDestY', 'ZIPd')
    plans_orig_to_des_zone = plans_orig_to_des[(plans_orig_to_des['ZIPo'].isin(zones))|(plans_orig_to_des['ZIPd'].isin(zones))]

    #READ AND ADD BLOCK ID TO EVENTS

    
    PTs = events_base[events_base['type']=='PathTraversal']
    
    PTs['duration'] = PTs['arrivalTime'] - PTs['departureTime']

    PTs['isCAV'] = PTs['vehicleType'].str.contains('L5')
    PTs['isRH'] = PTs['vehicle'].str.contains('rideHail')
    PTs['is_pool'] = PTs['numPassengers'] >1
    PTs['isRHpool'] = PTs['vehicle'].str.contains('rideHail')*PTs['is_pool']
    PTs['isBS'] = PTs['vehicle'].str.contains('bay_wheels')
    PTs['isRH_WC'] = PTs['vehicleType'].str.contains('RH_Car-wheelchair')
    PTs['is_empty'] = PTs['numPassengers'] == 0
    PTs['is_RHempty'] = PTs['isRH']*PTs['is_empty']
    PTs['is_car_emer'] = PTs['vehicle'].str.contains('emergency')
    
    PTs.loc[PTs['mode'] == 'car', 'capacity'] += 1
    PTs.loc[PTs['mode'] == 'car_hov2', 'capacity'] += 1
    PTs.loc[PTs['mode'] == 'car_hov3', 'capacity'] += 1
    PTs.loc[PTs['mode'] == 'walk', 'capacity'] = 1
    PTs.loc[PTs['mode'] == 'bike', 'capacity'] = 1
    
    PTs.loc[PTs['isRH'], 'mode'] += '_RideHail'
    PTs.loc[PTs['isRHpool'], 'mode'] += '_Pool'
    PTs.loc[PTs['isBS'], 'mode'] += '_Sharing'
    PTs.loc[PTs['isRH_WC'], 'mode'] += '_WC'
    PTs.loc[PTs['isCAV'], 'mode'] += '_CAV'
    PTs.loc[PTs['is_RHempty'], 'mode'] += '_empty'
    PTs.loc[PTs['is_car_emer'], 'mode'] += '_emer'
    
    PTs.loc[PTs['mode'] == 'car', 'occupancy'] = 1
    PTs.loc[PTs['mode'] == 'car_emer', 'occupancy'] = 1
    PTs.loc[PTs['mode'] == 'car_hov2', 'occupancy'] += 1
    PTs.loc[PTs['mode'] == 'car_hov3', 'occupancy'] += 1
    PTs.loc[PTs['mode'] == 'car_hov3_emer', 'occupancy'] = 1
    PTs.loc[PTs['mode'] == 'walk', 'occupancy'] = 1
    PTs.loc[PTs['mode'] == 'bike', 'occupancy'] = 1

    # PTs['vehicleMiles'] = PTs['length'] / 1609.34
    # PTs['passengerMiles'] = (PTs['length'] * PTs['occupancy']) / 1609.34
    # PTs['totalEnergyInJoules'] = PTs['primaryFuel'] + PTs['secondaryFuel']
    transit_modes = ['bus', 'subway', 'tram', 'rail','cable_car', 'ferry']
    for tm in transit_modes:
        PTs['is'+tm] = PTs['mode'].str.contains(tm)
    for tm in transit_modes:
        PTs['is_'+tm+'_empty'] = PTs['is'+tm]*PTs['is_empty']
    PTs['is_transit'] = 0
    for tm in transit_modes:
        PTs['is_transit']+=PTs['is'+tm]
    for tm in transit_modes:
        PTs.loc[PTs['is_'+tm+'_empty'], 'mode'] += '_empty'
        PTs.drop(columns=['is'+tm])
        PTs.drop(columns=['is_'+tm+'_empty'])
        
    PTs.drop(columns=['isCAV','is_empty','is_RHempty','isRH_WC','is_car_emer'])
    
    
    
    
    PTs = addGeometryIdToDataFrame2(PTs, BGs, 'startX', 'startY', 'ZIPo')
    PTs = addGeometryIdToDataFrame2(PTs, BGs, 'endX', 'endY', 'ZIPd')
    PTs = PTs[(PTs['ZIPo'].isin(zones))|(PTs['ZIPd'].isin(zones))]
    
    print(plans_orig_to_des)

    return plans_orig_to_des, PTs

In [6]:
#READ - DO IT FOR EACH SCENARIO
print('Baseline')
plans_base =     pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/beam/year-2020-iteration-4/ITERS/it.0/0.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'planElementIndex', 'legMode', 'legDepartureTime', 'activityEndTime', 'legTravelTime'])
events_base = pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz' , usecols = ['mode','type','startX', 'startY', 'endX', 'endY','length', 'departureTime', 'arrivalTime','vehicleType', 'vehicle', 'numPassengers', 'capacity'])
BGs = gpd.read_file('/vsicurl/https://github.com/LBNL-UCB-STI/beam-core-analysis/raw/main/Users/Zach/scenario/sfbay-blockgroups-2010/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')




Baseline


/tmp/ipykernel_1802928/1272599111.py:4: DtypeWarning: Columns (14,15,28) have mixed types. Specify dtype option on import or set low_memory=False.
  events_base = pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz' , usecols = ['mode','type','startX', 'startY', 'endX', 'endY','length', 'departureTime', 'arrivalTime','vehicleType', 'vehicle', 'numPassengers', 'capacity'])


In [8]:
#GET PLANS AND EVENTS FILTERED FOR THE CORDON ZONE
plans_orig_to_des, events_base_pt = process(plans_base, events_base, BGs, cordon_zones)
#GET PLANS AND EVENTS FILTERED FOR SF
# plans_orig_to_des, events_base_pt = process(plans_base, events_base, BGs, SF_zones)

#REPEAT FOR EACH SCENARIO

         personId  planSelected  planElementIndex activityType  \
7563273         4          True                 0         Home   
7563274         4          True                 1          NaN   
7563275         4          True                 2     othdiscr   
7563276         4          True                 3          NaN   
7563277         4          True                 4       social   
...           ...           ...               ...          ...   
2283162   6808834          True                 2         work   
2283163   6808834          True                 3          NaN   
2283164   6808834          True                 4       eatout   
2283165   6808834          True                 5          NaN   
2283166   6808834          True                 6         Home   

         activityLocationX  activityLocationY  activityEndTime legMode  \
7563273      567919.614977       4.189908e+06          55026.0     NaN   
7563274                NaN                NaN              

/opt/conda/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/conda/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/conda/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of 

          personId  planSelected  planElementIndex activityType  \
7563274          4          True                 1          NaN   
7563276          4          True                 3          NaN   
7563278          4          True                 5          NaN   
7563291          8          True                 1          NaN   
7563293          8          True                 3          NaN   
...            ...           ...               ...          ...   
11920788   6808782          True                 5          NaN   
11920790   6808782          True                 7          NaN   
2283161    6808834          True                 1          NaN   
2283163    6808834          True                 3          NaN   
2283165    6808834          True                 5          NaN   

          activityLocationX  activityLocationY  activityEndTime   legMode  \
7563274                 NaN                NaN              NaN       car   
7563276                 NaN              

In [32]:
plans_orig_to_des

,personId,planSelected,planElementIndex,activityType,activityLocationX,activityLocationY,activityEndTime,legMode,legDepartureTime,legTravelTime,tripOriginX,tripOriginY,tripDestX,tripDestY,ZIPo,ZIPd
7563274,4,True,1,NaN,NaN,NaN,NaN,car,55026.0,-inf,567919.614977,4.189908e+06,566680.335629,4.183202e+06,060014001001,060014054022
7563276,4,True,3,NaN,NaN,NaN,NaN,car,58204.8,-inf,566680.335629,4.183202e+06,565948.027887,4.185053e+06,060014054022,060014037021
7563278,4,True,5,NaN,NaN,NaN,NaN,car,59457.6,-inf,565948.027887,4.185053e+06,567919.614977,4.189908e+06,060014037021,060014001001
7563291,8,True,1,NaN,NaN,NaN,NaN,car,56210.4,-inf,567919.614977,4.189908e+06,564199.236050,4.191804e+06,060014001001,060014229002
7563293,8,True,3,NaN,NaN,NaN,NaN,car_hov2,73836.0,-inf,564199.236050,4.191804e+06,567919.614977,4.189908e+06,060014229002,060014001001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11920788,6808782,True,5,NaN,NaN,NaN,NaN,car,60026.4,-inf,500118.780671,4.251865e+06,521057.276130,4.255188e+06,060971543043,060971530063
11920790,6808782,True,7,NaN,NaN,NaN,NaN,car,75333.6,-inf,521057.276130,4.255188e+06,500118.780671,4.251865e+06,060971530063,060971543043
2283161,6808834,True,1,NaN,NaN,NaN,NaN,car,26589.6,-inf,494885.497981,4.242350e+06,516981.693500,4.263965e+06,060971543043,060971527021
2283163,6808834,True,3,NaN,NaN,NaN,NaN,car,56790.0,-inf,516981.693500,4.263965e+06,526488.631550,4.256155e+06,060971527021,060971523004


In [33]:
events_base_pt

,type,vehicle,vehicleType,startX,startY,endX,endY,mode,length,arrivalTime,...,isferry,is_bus_empty,is_subway_empty,is_tram_empty,is_rail_empty,is_cable_car_empty,is_ferry_empty,is_transit,ZIPo,ZIPd
880370,PathTraversal,AC:5572515,BUS-AC,-122.272623,37.803171,-122.422633,37.772372,bus_empty,18651.739000,3080.0,...,False,True,False,False,False,False,False,1,060014031001,060750202001
880497,PathTraversal,AC:5572515,BUS-AC,-122.422633,37.772372,-122.418675,37.775550,bus_empty,414.315000,3600.0,...,False,True,False,False,False,False,False,1,060750202001,060750177002
880572,PathTraversal,AC:5572516,BUS-AC,-122.419111,37.775222,-122.413206,37.779879,bus_empty,679.923000,3900.0,...,False,True,False,False,False,False,False,1,060750177002,060750176013
880632,PathTraversal,AC:5572516,BUS-AC,-122.414715,37.778681,-122.407357,37.784577,bus_empty,732.446000,4140.0,...,False,True,False,False,False,False,False,1,060750176013,060750176011
880643,PathTraversal,AC:5572516,BUS-AC,-122.407528,37.784441,-122.403385,37.787624,bus_empty,494.381000,4230.0,...,False,True,False,False,False,False,False,1,060750176011,060750615001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36795133,PathTraversal,body-2727310,BODY-TYPE-DEFAULT,-122.396168,37.796828,-122.271080,38.265720,walk,55247.025776,199135.0,...,False,False,False,False,False,False,False,0,060750105002,060552009001
36795220,PathTraversal,body-5949578,BODY-TYPE-DEFAULT,-122.406506,37.792549,-122.124654,37.436302,walk,51820.709361,201162.0,...,False,False,False,False,False,False,False,0,060750118001,060855110002
36795340,PathTraversal,body-2466635,BODY-TYPE-DEFAULT,-122.417888,37.622358,-122.402755,37.789709,walk,18782.693373,205777.0,...,False,False,False,False,False,False,False,0,060816040002,060750117002
36795364,PathTraversal,body-6175723,BODY-TYPE-DEFAULT,-122.404205,37.786517,-121.852938,38.448209,walk,97564.802839,207601.0,...,False,False,False,False,False,False,False,0,060750615001,060952534033


In [11]:
plans_orig_to_des.legMode.value_counts()

car                   1198829
car_hov2               240739
walk                   226051
hov2_teleportation     212053
walk_transit           199718
hov3_teleportation     189646
car_hov3               172830
bike                    40070
ride_hail               39397
ride_hail_pooled        13971
drive_transit            1847
bike_transit                1
Name: legMode, dtype: int64

In [34]:
events_base_pt = events_base_pt[events_base_pt['length']>0]
events_base_pt['duration'] = events_base_pt['arrivalTime']-events_base_pt['departureTime']
grouped = events_base_pt.groupby('mode')
vmt_mode = grouped['length'].sum()
vmt_mode_av = grouped['length'].mean()
vht_mode = grouped['duration'].sum()
vht_mode_av = grouped['duration'].mean()
speed_mode = grouped['length'].sum()/grouped['duration'].sum()


/tmp/ipykernel_1802928/2310822191.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_base_pt['duration'] = events_base_pt['arrivalTime']-events_base_pt['departureTime']


In [35]:
vmt_mode

mode
bike                  2.224894e+07
bus                   3.097541e+07
bus_empty             2.876143e+07
cable_car             1.021272e+06
cable_car_empty       1.106097e+06
car                   7.658455e+08
car_RideHail          2.666463e+07
car_RideHail_Pool     1.791996e+05
car_RideHail_empty    5.845625e+06
car_emer              3.245398e+07
car_hov2              1.014784e+08
car_hov2_emer         7.773377e+06
car_hov3              7.597274e+07
car_hov3_emer         5.397994e+06
ferry                 1.905292e+05
ferry_empty           6.689995e+05
rail                  5.943247e+05
rail_empty            3.147116e+04
subway                7.780653e+06
subway_empty          3.231714e+05
tram                  5.924649e+06
tram_empty            2.952318e+06
walk                  2.041749e+08
Name: length, dtype: float64

In [36]:
vmt_mode_av

mode
bike                   3522.073334
bus                     539.829449
bus_empty               507.300914
cable_car               118.052440
cable_car_empty         120.424262
car                    8247.224551
car_RideHail           8867.518924
car_RideHail_Pool      3446.146750
car_RideHail_empty     2317.852847
car_emer               8121.617301
car_hov2               9368.387342
car_hov2_emer         10518.777893
car_hov3               9082.215874
car_hov3_emer         10033.445197
ferry                 31754.859743
ferry_empty           11736.832916
rail                   7718.502293
rail_empty             2098.077469
subway                 2399.954803
subway_empty           2485.933833
tram                    591.695701
tram_empty              424.427535
walk                   1208.793305
Name: length, dtype: float64

In [37]:
vht_mode

mode
bike                    4449791.0
bus                     5662413.0
bus_empty               5955187.0
cable_car                416631.0
cable_car_empty          413889.0
car                    41787098.0
car_RideHail            1454164.0
car_RideHail_Pool         10713.0
car_RideHail_empty       347697.0
car_emer                1712950.0
car_hov2                5513328.0
car_hov2_emer            402997.0
car_hov3                4190615.0
car_hov3_emer            291409.0
ferry                     20100.0
ferry_empty               96000.0
rail                      44700.0
rail_empty                 4920.0
subway                   505620.0
subway_empty              20700.0
tram                    1179721.0
tram_empty               801150.0
walk                  152654538.0
Name: duration, dtype: float64

In [38]:
vht_mode_av

mode
bike                   704.415229
bus                     98.682694
bus_empty              105.039016
cable_car               48.159866
cable_car_empty         45.061404
car                    449.996209
car_RideHail           483.592950
car_RideHail_Pool      206.019231
car_RideHail_empty     137.865583
car_emer               428.666166
car_hov2               508.985229
car_hov2_emer          545.327470
car_hov3               500.970114
car_hov3_emer          541.652416
ferry                 3350.000000
ferry_empty           1684.210526
rail                   580.519481
rail_empty             328.000000
subway                 155.959284
subway_empty           159.230769
tram                   117.818935
tram_empty             115.173951
walk                   903.773285
Name: duration, dtype: float64

In [39]:
speed_mode

mode
bike                   4.999996
bus                    5.470356
bus_empty              4.829643
cable_car              2.451262
cable_car_empty        2.672448
car                   18.327320
car_RideHail          18.336742
car_RideHail_Pool     16.727306
car_RideHail_empty    16.812411
car_emer              18.946252
car_hov2              18.406010
car_hov2_emer         19.288920
car_hov3              18.129257
car_hov3_emer         18.523771
ferry                  9.479063
ferry_empty            6.968745
rail                  13.295854
rail_empty             6.396578
subway                15.388342
subway_empty          15.612145
tram                   5.022076
tram_empty             3.685100
walk                   1.337496
dtype: float64

In [44]:
events_base = pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz' , nrows=3000000, )


/tmp/ipykernel_1802928/2647526998.py:1: DtypeWarning: Columns (0,2,6,14,15,20,21,27,28,34,36,38,41,42,43,44,46,50,53,59,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  events_base = pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz' , nrows=3000000, )


In [42]:
events_base.keys()

Index(['person', 'link', 'legMode', 'time', 'type', 'primaryFuelLevel',
       'parkingTaz', 'chargingPointType', 'pricingModel', 'parkingType',
       'locationY', 'locationX', 'price', 'secondaryFuelLevel', 'vehicle',
       'vehicleType', 'shiftStatus', 'parkingZoneId', 'fuel', 'duration',
       'actType', 'reason', 'startX', 'startY', 'endX', 'endY', 'score',
       'driver', 'mode', 'incentive', 'tollCost', 'netCost', 'tripId', 'cost',
       'currentTourMode', 'expectedMaximumUtility', 'availableAlternatives',
       'location', 'personalVehicleAvailable', 'length', 'tourIndex',
       'legModes', 'legVehicleIds', 'currentActivity', 'nextActivity',
       'departTime', 'requireWheelchair', 'arrivalTime', 'departureTime',
       'facility', 'links', 'numPassengers', 'primaryFuel', 'riders',
       'toStopIndex', 'fromStopIndex', 'seatingCapacity', 'tollPaid',
       'capacity', 'linkTravelTime', 'secondaryFuel', 'secondaryFuelType',
       'primaryFuelType'],
      dtype='object'

In [46]:
events_base['fuel'].value_counts()

34600000.0    1
36650000.0    1
750000.0      1
54200000.0    1
4950000.0     1
29600000.0    1
Name: fuel, dtype: int64

In [47]:
events_base['primaryFuel'].value_counts()

0.000000e+00    182865
3.483881e+06      1152
6.154096e+07       827
1.055491e+08       701
1.285277e+05       522
                 ...  
3.270292e+07         1
5.745512e+07         1
7.741339e+03         1
2.423430e+04         1
1.879539e+03         1
Name: primaryFuel, Length: 191321, dtype: int64

In [48]:
events_base['secondaryFuel'].value_counts()

0.000000e+00    545381
4.722079e+06         1
2.589839e+07         1
1.338294e+07         1
9.576245e+04         1
1.370789e+07         1
4.502687e+07         1
2.059717e+05         1
8.062716e+05         1
1.647518e+07         1
7.776786e+06         1
2.397878e+06         1
1.440370e+05         1
4.687851e+06         1
3.095444e+04         1
1.898117e+07         1
6.996824e+04         1
4.001412e+07         1
2.232090e+05         1
2.463290e+07         1
3.032908e+04         1
6.090164e+04         1
Name: secondaryFuel, dtype: int64

In [49]:
events_base['primaryFuelType'].value_counts()

Food           234972
Gasoline       193413
Diesel          89562
Biodiesel       18567
Electricity      8888
Name: primaryFuelType, dtype: int64

In [50]:
events_base['secondaryFuelType'].value_counts()

None        544982
Gasoline       420
Name: secondaryFuelType, dtype: int64